In [1]:
import loss_landscapes
import torch
from HGNN.train import CNN, dataLoader
from HGNN.train.configParser import ConfigParser
import os
import numpy as np
from tqdm.auto import tqdm 

In [2]:
cuda=6

# set cuda
if torch.cuda.is_available():
    torch.cuda.set_device(cuda)
    print("using cuda", cuda)

using cuda 6


In [3]:
# experiment_params = {
#             "image_path": "INHS_cropped",
#             "suffix": "biology_paper_cleaned_200max",
#             "img_res": 448,
#             "augmented": True,
#             "batchSize": 128,
#             "learning_rate": 0.1,
#             "numOfTrials": 1,
#             "fc_layers": 1,
#             "modelType": "BB",
#             "lambda": 0.01,
#             "unsupervisedOnTest": False,
#             "tl_model": "ResNet18",
#             "link_layer": "avgpool"
#         }
# model_path = "/home/elhamod/HGNN/experiments/biology_paper_surfaceExperiments_BB_lr_largeBatch/models/8d50d7c9daa5d26371050c2e66f8cb6fb23a94d007db9dbc8bc48770"
# experimentsPath = "/home/elhamod/HGNN/experiments/" 
# experimentName = "biology_paper_surfaceExperiments_BB_lr_largeBatch"
# dataPath = "/data/BGNN_data"


# coord_Tracker_path = "/home/elhamod/HGNN/coordinates/biology_paper_surfaceExperiments_BB_lr_largeBatch_01"

experiment_params = {"image_path": "INHS_cropped", "suffix": "biology_paper_cleaned_15max", "img_res": 448, "augmented": True, "batchSize": 64, "learning_rate": 0.0001, "numOfTrials": 1, "fc_layers": 1, "modelType": "HGNN_add", "lambda": 0.01, "unsupervisedOnTest": False, "tl_model": "ResNet18", "link_layer": "avgpool"}
model_path = "/home/elhamod/HGNN/experiments/biology_paper_surfaceExperiments_HGNN_datasets/models/af109a6b3598cfb56a4f87b4ab1167d63ec855654e268e5ebb3f2593"
experimentsPath = "/home/elhamod/HGNN/experiments/" 
experimentName = "biology_paper_surfaceExperiments_HGNN_datasets"
dataPath = "/data/BGNN_data"


coord_Tracker_path = "/home/elhamod/HGNN/coordinates/testing3"

normalize = 'filter'
Do_pca = True
steps = 25
# grid_batchSize = 200

In [4]:
model_paths = []

i=0
while(True):
    p = os.path.join(model_path, "iterations", "iteration{0}.pt".format(i))
    if os.path.exists(p):
        model_paths.append(p)
        i = i+1
    else:
        model_paths.append(os.path.join(model_path, "iterations", "finalModel.pt"))
        print(i, " models added")
        break

38  models added


In [5]:
experimentPathAndName = os.path.join(experimentsPath, experimentName)
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)
datasetManager.updateParams(config_parser.fixPaths(experiment_params))
train_loader, validation_loader, test_loader = datasetManager.getLoaders()
architecture = {
    "fine": len(train_loader.dataset.csv_processor.getFineList()),
    "coarse" : len(train_loader.dataset.csv_processor.getCoarseList())
}

Creating datasets...


scanning files: 100%|██████████| 570/570 [00:02<00:00, 275.41it/s]

Creating datasets... Done.
Creating loaders...
Creating loaders... Done.


In [6]:
models = []
for model_path in tqdm(model_paths):
    model = CNN.create_model(architecture, experiment_params)
    model.load_state_dict(torch.load(os.path.join(model_path))) # , map_location=torch.device('cpu')
    model.eval()
    models.append(model)

In [7]:
# print(list(models))

In [8]:
print(len(models))

39


In [9]:
# wrapped_model_params = []
# for i in models:
#     wrapped_model_params.append(loss_landscapes.wrap_model(i).get_module_parameters())

In [10]:
# a, b = loss_landscapes.get_best_fit_orthogonal_bases_pca(wrapped_model_params)

In [11]:
# a.as_numpy()

In [12]:
# b.as_numpy()

In [13]:
# import time

class BGNN_loss(loss_landscapes.Metric):
    def __init__(self, loader, lambda_, isDSN):
        self.criterion = torch.nn.CrossEntropyLoss()
        self.isDSN = isDSN
        self.lambda_ = lambda_
        self.loader = loader
        
        self.inputs = []
        self.target = []
        self.meta_target = []
        self.index = -1
        
    def get_next(self):
        if len(self.inputs) == 0:
            for batch in self.loader:
                self.inputs.append(batch["image"])
                self.target.append(batch["fine"])
                self.meta_target.append(batch["coarse"])
        self.index = self.index+1
        if self.index==len(self.inputs):
            self.index=-1
            return None, None, None
        return self.inputs[self.index], self.target[self.index], self.meta_target[self.index]

    def __call__(self, model_wrapper) -> float:
#         s = time.time()
        
        inputs, target, meta_target = self.get_next()
        while inputs is not None:
            if torch.cuda.is_available():
                self.criterion = self.criterion.cuda()
                inputs = inputs.cuda()
                target = target.cuda()
                meta_target = meta_target.cuda()
            
            batch_size = inputs.shape[0]
                
            output = model_wrapper.forward(inputs)
            print(output)
            
            loss_coarse = 0
            if output["coarse"] is not None:
                loss_coarse = self.criterion(output["coarse"], meta_target if not self.isDSN else target)
            loss_fine = self.criterion(output["fine"], target)
            loss = loss_fine + self.lambda_*loss_coarse
            loss = batch_size*loss
            
            inputs, target, meta_target = self.get_next()
            
#             print(time.time()-s)
#             s = time.time()

        print(len(self.loader.dataset))
        loss = loss/len(self.loader.dataset)
        loss = loss.detach()
        print(loss)
        print("--")
        if torch.cuda.is_available():
            loss = loss.cpu()
        
        return loss.numpy()

In [14]:
import time

class BGNN_loss3(loss_landscapes.Metric):
    def __init__(self, loader, lambda_, isDSN):
        self.criterion = torch.nn.CrossEntropyLoss()
        self.isDSN = isDSN
        self.lambda_ = lambda_
        self.loader = loader
        self.data_iter = iter(loader)

    def __call__(self, model_wrapper) -> float:
        s = time.time()
        
        next_batch = self.data_iter.next() # start loading the first batch
#         print(next_batch)
#         next_batch = [ _.cuda(non_blocking=True) for _ in next_batch ]  # with pin_memory=True and non_blocking=True, this will copy data to GPU non blockingly

        for i in range(len(self.loader)):
            batch = next_batch 
            if i + 1 != len(self.loader): 
                # start copying data of next batch
                next_batch = self.data_iter.next()
#                 next_batch = [ _.cuda(async=True) for _ in next_batch]

            inputs = batch["image"]
            target = batch["fine"]
            meta_target = batch["coarse"]
            if torch.cuda.is_available():
                self.criterion = self.criterion.cuda()
                inputs = inputs.cuda()
                target = target.cuda()
                meta_target = meta_target.cuda()
            
            batch_size = inputs.shape[0]
                
            
            output = model_wrapper.forward(inputs)
            
            
            loss_coarse = 0
            if output["coarse"] is not None:
                loss_coarse = self.criterion(output["coarse"], meta_target if not self.isDSN else target)
            loss_fine = self.criterion(output["fine"], target)
            loss = loss_fine + self.lambda_*loss_coarse
            loss = batch_size*loss
            
            print(time.time()-s)
            s = time.time()

        loss = loss/len(self.loader.dataset)
        loss = loss.detach()
        if torch.cuda.is_available():
            loss = loss.cpu()
        
        return loss.numpy()

In [15]:
class BGNN_loss2(loss_landscapes.Loss):
    def __init__(self, inputs, target, meta_target, lambda_, isDSN):
        criterion = torch.nn.CrossEntropyLoss()
        if torch.cuda.is_available():
            criterion = criterion.cuda()
            inputs = inputs.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
        super().__init__(criterion, inputs, target)
        self.isDSN = isDSN
        self.lambda_ = lambda_
        self.meta_target = meta_target

    def __call__(self, model_wrapper) -> float:
        output = model_wrapper.forward(self.inputs)
        
        loss_coarse = 0
        if output["coarse"] is not None:
            loss_coarse = self.loss_fn(output["coarse"], self.meta_target if not self.isDSN else self.target)
        loss_fine = self.loss_fn(output["fine"], self.target)
        loss = loss_fine + self.lambda_*loss_coarse
        
        loss = loss.detach()
        if torch.cuda.is_available():
            loss = loss.cpu()
        
        return loss.numpy()

In [16]:
# grid_batchSize = len(test_loader.dataset)

# num_of_workers = 8
# test_loader = torch.utils.data.DataLoader(test_loader.dataset, batch_size=grid_batchSize, num_workers=num_of_workers, pin_memory=True)
# batch = next(iter(test_loader))
# inputs = batch["image"]
# target = batch["fine"]
# meta_target = batch["coarse"]

In [17]:
test_loader = torch.utils.data.DataLoader(test_loader.dataset, batch_size=len(test_loader.dataset))
batch = next(iter(test_loader))
inputs = batch["image"]
target = batch["fine"]
meta_target = batch["coarse"]

In [18]:
# metric = BGNN_loss3(test_loader, experiment_params["lambda"], False) #TODO: need to automate these params
# metric = BGNN_loss(test_loader, experiment_params["lambda"], False) #TODO: need to automate these params
metric = BGNN_loss2(inputs, target, meta_target, experiment_params["lambda"], False) #TODO: need to automate these params

In [19]:
coord_Tracker = loss_landscapes.Coordinates_tracker(path=coord_Tracker_path)

In [20]:
# print(coord_Tracker.dirs_[0].model_norm() )
# print(coord_Tracker.scaled_dirs_[0].model_norm() )
# print(loss_landscapes.wrap_model(models[-1]).get_module_parameters().model_norm())
# print(coord_Tracker.dist_)

In [22]:
# for l in range(len(coord_Tracker.dirs_[0].parameters)):
#     print(coord_Tracker.dirs_[0].parameters[l].shape)
#     print(len(coord_Tracker.dirs_[0].parameters[l].size()))
#     print(len(coord_Tracker.dirs_[0].parameters[l]))
#     # normalize one-dimensional bias vectors
#     if len(coord_Tracker.dirs_[0].parameters[l].size()) == 1:
#         print('1-')
# #         self.parameters[l] *= (ref_point.parameters[l].norm(order) / (self.parameters[l].norm(order)+ sys.float_info.epsilon))
#     # normalize two-dimensional weight vectors
#     for f in range(len(coord_Tracker.dirs_[0].parameters[l])):
#         print('2-', f)
# #         self.parameters[l][f] *= ref_point.filter_norm((l, f), order) / (self.filter_norm((l, f), order)+ sys.float_info.epsilon)
#     print("---")

In [23]:
if Do_pca:
    if not coord_Tracker.load():
        coord_Tracker.update_directions(loss_landscapes.get_best_fit_orthogonal_bases_pca(models))
        coord_Tracker.update_distance(loss_landscapes.get_optimal_distance(
            models[:-1], 
            models[-1]))
        coord_Tracker.save()
        print(coord_Tracker.dist_)
        print("saved")
    else:
        print("loaded")

PCA time:  57.033881425857544
1.4936075e-08
5.224507808685303
saved


In [24]:
# a = coord_Tracker.dirs_[0].as_numpy()
# print(a)
# b = loss_landscapes.numpy_to_ModelParameters(a, coord_Tracker.dirs_[0])
# print(b.as_numpy())

[-4.17232332e-05 -1.45999475e-05 -1.87032438e-05 ... -1.36305497e-03
 -1.27225055e-03 -1.44457351e-03]
[-4.17232332e-05 -1.45999475e-05 -1.87032438e-05 ... -1.36305497e-03
 -1.27225055e-03 -1.44457351e-03]


In [25]:
loss_data_file = os.path.join(coord_Tracker_path, "loss_surface_data")
# try:
#     loss_data = np.load(loss_data_file)
#     print("loaded")
# except:
loss_data = loss_landscapes.random_plane(models[-1], metric, steps=steps, distance=coord_Tracker.dist_, normalization=normalize, deepcopy_model=True, coord_Tracker=coord_Tracker)
np.save(loss_data_file, loss_data)
print("saved")

ZeroDivisionError: float division by zero

In [ ]:
# metric(loss_landscapes.wrap_model(models[0]))

In [ ]:
# print(loss_data.shape)

In [ ]:
def scale_model(model_params, center_model_params, dirs_):
    dir_one = dirs_[0]
    dir_two = dirs_[1]

    model_params_one = model_params.dot(dir_one)
    model_params_two = model_params.dot(dir_two)
    
    center_model_params_one = center_model_params.dot(dir_one)
    center_model_params_two = center_model_params.dot(dir_two)

    return model_params_one - center_model_params_one, model_params_two - center_model_params_two

In [ ]:
import matplotlib.pyplot as plt
from myhelpers import config_plots
config_plots.global_settings()

num_rows=1
num_columns=1
STEPS = coord_Tracker.steps_
DISTANCE = coord_Tracker.dist_

f, axes = plt.subplots(num_rows, num_columns, figsize=(40, 15), dpi= 300,)

X = np.array([(k - int(STEPS/2))*DISTANCE/STEPS for k in range(STEPS)])
Y = X
loss_data_fin = loss_data

ax = plt.subplot(num_rows, num_columns, 1)
contours = plt.contour(X, Y, loss_data_fin, 15, colors='black') # 
plt.clabel(contours, inline=True, fontsize=10)
plt.imshow(loss_data_fin, extent=[X[0], X[-1], Y[0], Y[-1]], origin='lower') # , cmap='RdGy', alpha=0.5
maxVal = np.amax([loss_data_fin])
minVal = np.amin([loss_data_fin])
plt.pcolor(X, Y, loss_data_fin, vmin=minVal, vmax=maxVal, cmap='Reds')
plt.colorbar()

ax.title.set_text('surface')
plt.plot([0], [0], marker='o', markersize=4, color="blue")

wm_center = loss_landscapes.wrap_model(models[-1]).get_module_parameters()

# for i in range(len(models)):
#     wm = loss_landscapes.wrap_model(models[i]).get_module_parameters()
#     scaled_model = scale_model(wm, wm_center, coord_Tracker.scaled_dirs_)
#     print(scaled_model)
#     plt.plot(scaled_model[0], scaled_model[1], marker='o', markersize=4, color="blue")

In [ ]:
f.savefig(os.path.join(coord_Tracker_path, "surface_fig.pdf"))